# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# TD4 - Régression linéaire

Ce TD concerne la régression linéaire (Chapitre 2 du cours). À la fin de ce TD, vous devriez être avoir été en mesure de :
- Choisir un modèle sur la base de l'estimation ou de la prédiction.
- Vérifier pour la présence de multicolinéarité.
- Transformer les variables pour modéliser des relations non-linéaires.


## Contexte : Poids des poissons

Le fichier *fishweights.csv* contient les caractéristiques de 158 poissons péchés dans le lac Laengelmavesi en Finlande. Nous étudierons le poids (Weight en grammes) des poissons en fonction de variables explicatives suivantes :
- l'espèce de poisson (species) ;
- la longueur standard (StandardLength) ;
- la longeur non standard  (NonStandardLength) ;
- la longueur totale (TotalLength) ;
- la hauteur (Height) ;
- la largeur (Width).


          ___/////___                  _
         /           \    ___          |
       /\             \_ /  /          H
     <   )            __)  \           |
       \/_\\_________/   \__\          _

     |------- SL -------|
     |------- NSL ---------|
     |------- TL ------------|

In [ ]:
using CSV 
using DataFrames
using Distributions
using Gadfly
using LinearAlgebra
using Statistics

## Chargement des données

In [ ]:
data = CSV.read("fishweights.csv", DataFrame)
first(data,10)

# Exercice 1 - Analyse exploratoire

## a) Illustrez les poids en fonction des espèces de poisson

**Note :** Avec Gadfly, la variable *x* peut être une variable catégorielle.

In [ ]:
Gadfly.set_default_plot_size(16cm, 9cm)
plot(data, x=:Species, y=:Weight, Geom.point)

## b) Illustrez les poids en fonction de la longeur standard et des espèces.

**Suggestion :** Utilisez l'option *color* de Gadfly pour distinguer les différentes espèces.

In [ ]:
Gadfly.set_default_plot_size(16cm, 9cm)
plot(data, x=:StandardLength, y=:Weight, Geom.point, color=:Species)

## c) Calculez le poids moyen pour chaque espèce ainsi que le nombre d'observations pour chaque espèce.

Les fonctions `combine()` et `groupby()` permettent d'effectuer rapidement ces tâches. Puisqu'elles ne sont pas intuitives, je vous donne d'emblée la réponse. C'est une combinaison de commandes qui pourra vous être utile dans tout le cours.

In [ ]:
df = combine(Weight = :Weight => mean, n=:Weight => length, groupby(data, :Species))

In [ ]:
sort!(df, :Weight)

# Exercice 2 - Ajustement *vs* prédiction

In [ ]:
"""
    construct_structure(x::Vector{<:Real}, order::Int)

Construction de la matrice de structure du modèle polynomial d'ordre `order` à partir du vecteur `x`.
"""
function construct_structure(x::Vector{<:Real}, order::Int)
    
    X = Array{Float64}(undef, length(x), order+1)
    
    for p in 0:order
       X[:,p+1] = x.^p 
    end
    
    return X
    
end

## a) Construction de l'ensemble d'entraînement et de validation pour les éperlans

In [ ]:
eperlan = filter(row -> row.Species == "Éperlan", data)
train = eperlan[1:2:end, :]
validation = eperlan[2:2:end, :]

## b) Modèle unidimensionnel

1. Estimez la droite de régression avec les données de l'ensemble d'entraînement.
2. Tracez la droite de régression ainsi que les points de l'ensemble d'entraînement.
3. Ajoutez les points de validation sur le graphique avec une autre couleur.

**Suggestion :** Utilisez la fonction `layer()` de Gadfly pour ajouter plusieurs couches à un graphique.

In [ ]:
order = 1

X = construct_structure(train.StandardLength, order)
β̂ = X\train.Weight

In [ ]:
xx = collect(range(9, stop=14, length = 100))
XX = construct_structure(xx, order)
yy = XX*β̂


train_layer = layer(train, x=:StandardLength, y=:Weight, Geom.point)
validation_layer = layer(validation, x=:StandardLength, y=:Weight, Geom.point, Theme(default_color ="red"))
model_layer = layer(x=xx, y=yy, Geom.line)

Gadfly.set_default_plot_size(16cm, 9cm)
plot(train_layer, validation_layer ,model_layer)

## c) Modèle cubique

1. Estimez les paramètres du modèle cubique en utilisant les données de l'ensemble d'entraînement.
2. Tracez la courbe donnée par le modèle ainsi que les points de l'ensemble d'entraînement.
3. Ajoutez les points de validation sur le graphique avec une autre couleur.

**Suggestion :** Utilisez la fonction `construct_structure()` fournie pour construire la matrice de structure pour le modèle cubique avec `order=3`.

In [ ]:
order = 3

X = construct_structure(train.StandardLength, order)
β̂ = X\train.Weight

In [ ]:
xx = collect(range(9, stop=14, length = 100))
XX = construct_structure(xx, order)
yy = XX*β̂


train_layer = layer(train, x=:StandardLength, y=:Weight, Geom.point)
validation_layer = layer(validation, x=:StandardLength, y=:Weight, Geom.point, Theme(default_color ="red"))
model_layer = layer(x=xx, y=yy, Geom.line)

Gadfly.set_default_plot_size(16cm, 9cm)
plot(train_layer, validation_layer ,model_layer)

## d) Modèle d'ordre 4

1. Estimez les paramètres du modèle d'ordre 4 en utilisant les données de l'ensemble d'entraînement.
2. Tracez la courbe donnée par le modèle ainsi que les points de l'ensemble d'entraînement.
3. Ajoutez les points de validation sur le graphique avec une autre couleur.

**Suggestion :** Utilisez la fonction `construct_structure()` fournie pour construire la matrice de structure pour le modèle cubique avec `order=4`.

In [ ]:
order = 4

X = construct_structure(train.StandardLength, order)
β̂ = X\train.Weight

In [ ]:
xx = collect(range(9, stop=14, length = 100))
XX = construct_structure(xx, order)
yy = XX*β̂


train_layer = layer(train, x=:StandardLength, y=:Weight, Geom.point)
validation_layer = layer(validation, x=:StandardLength, y=:Weight, Geom.point, Theme(default_color ="red"))
model_layer = layer(x=xx, y=yy, Geom.line)

Gadfly.set_default_plot_size(16cm, 9cm)
plot(train_layer, validation_layer ,model_layer)

## e) Calcul de la qualité d'ajustement et de l'erreur de prédiction

Pour les modèles d'ordre 0 à 6, calculez 
- le coefficient de détermination
- le coefficient de détermination
- l'erreur quadratique moyenne (MSE) sur l'échantillon de validation.

**Suggestion :** Ajoutez un ligne au tableau `df` proposé pour chacun des ordre de modèle avec la fonction `push!()`.


In [ ]:
df = DataFrame(Ordre = Int64[], R² = Float64[], R²_aj = Float64[], MSE = Float64[])

SST = sum( (train.Weight .- mean(train.Weight)).^2)

n = length(train.Weight)

for order in 0:6

    X = construct_structure(train.StandardLength, order)
    β̂ = X\train.Weight

    e = train.Weight - X*β̂

    SSE = e'e

    X₀ = construct_structure(validation.StandardLength, order)
    ŷ₀ = X₀*β̂
    e₀ = validation.Weight - ŷ₀
    
    MSE = mean(e₀.^2)
    
    push!(df, [order, 1 -  SSE/SST, 1 - (n-1)/(n-order-1)*SSE/SST, MSE])
    
end

df


In [ ]:
Gadfly.set_default_plot_size(21cm, 16cm)
fig1 = plot(df, x=:Ordre, y=:R², Geom.line, Geom.point, Coord.cartesian(ymin=0))
fig2 = plot(df, x=:Ordre, y=:R²_aj, Geom.line, Geom.point, Coord.cartesian(ymin=0))
fig3 = plot(df, x=:Ordre, y=:MSE, Geom.line, Geom.point, Coord.cartesian(ymin=0, ymax=5))
gridstack([fig1 fig2 ; fig3 plot()])

## f) Sélection du meilleur modèle

Selon vous, quel est le meilleur modèle ?

**Du point de vue de l'estimation**

Si on se fie au coefficient de détermination ajusté, alors le modèle qui s'ajuste le mieux aux données est le modèle quadratique. C'est en effet le maximum local du coefficient de détermination ajusté pour les modèles simples. Pour les ordres supérieurs, on est en surapprentissage.

**Du point de vue de la prédiction**

Si on se fie à l'erreur quadratique moyenne, le modèle linéaire est le meilleur pour prédire le poids des éperlans en fonction de leur longueur.

**Conclusion**

Le modèle s'ajustant le mieux aux données est le modèle quadratique et le modèle donnant de meilleurs prédictions au sens de l'erreur quadratique moyenne est le modèle linéaire. Il n'y a donc pas une seule réponse possible. Le meilleur modèle dépendra de l'application.

# Exercice 3 - Poids des perches

On sélectionne pour cet exercice que les perches.

In [ ]:
perche = filter(row -> row.Species == "Perche", data)
n = size(perche,1)
first(perche, 10)

## a) Vérification quantitative de multicolinéarité.

Calculez le facteur d'inflation de la variance VIF pour les variables explicatives de la perche. Déterminez s'il y a multicolinéarité.

In [ ]:
# Matrice de structure

X = hcat(ones(n), convert(Array{Float64}, perche[:, 3:end]))

In [ ]:
function compute_VIF(structureMatrix::Array{T,2} where T<:Real)
    
    n, m = size(structureMatrix)
    
    p = m-1  # nb de variables explicatives
    
    VIF = Float64[]
    
    for j in 2:m
       
        y = structureMatrix[:,j]
        X = structureMatrix[:, setdiff(1:m, j)]
        
        β̂ = X\y
        
        e = y - X*β̂
        
        SST = sum( (y .- mean(y)).^2)
        SSE = e'e
        
        R² = 1 - SSE/SST
        
        push!(VIF, 1/(1-R²))
        
    end
    
    return VIF
    
end

In [ ]:
compute_VIF(X)

## b) Vérification visuelle de la multicolinéarité

Illustrer la :NonStandardLength en fonction de la :StandardLength. Est-ce que ça corrobore le fait que la multicolinéarité ait été détectée à la question précédente.

In [ ]:
Gadfly.set_default_plot_size(16cm, 9cm)
plot(perche, x=:StandardLength, y=:NonStandardLength, Geom.point)

## c)  Identification du meilleur modèle

Puisqu'il y a présence de multicolinéarité entre toutes les variables, on cherche le meilleur modèle unidimensionnel.

**Suggestion :** Transformez la variable d'intérêt pour trouver le meilleur modèle unidimensionnel.

In [ ]:
varnames = [:StandardLength, :NonStandardLength, :TotalLength, :Height, :Width]

perche[!,:Weight_csquared] = (perche[:,:Weight]).^(1/3)

y = perche.Weight_csquared

n = length(y)

SST = sum( (y .- mean(y)).^2 )

res = DataFrame(Variable = Symbol[], R² = Float64[])

for varname in varnames
   
    X = hcat(ones(n), perche[:,varname])
    β̂ = X\y
    
    e = y - X*β̂
    SSE = e'e
    
    push!(res, [varname, 1-SSE/SST])
    
end

sort!(res, :R², rev=true)

In [ ]:
x = perche[:,res.Variable[1]]
y = perche.Weight_csquared

n = length(y)

X = hcat(ones(n), x)

β̂ = X\y

Gadfly.set_default_plot_size(16cm, 9cm)
plot(x=x, y=y, Geom.point, intercept = [β̂[1]], slope = [β̂[2]], Geom.abline(color="red", style=:dash))

## d) Vérification des hypothèses de la régression

Vérifiez si les hypothèses 1 à 4 de la régression sont satisfaites. Pour ce faire, tracer les graphiques suivants :
- le nuage des points {(ŷᵢ, eᵢ) : 1 ≤ i ≤ n} pour vérifier les hypothèses 1 et 2
- la droite de Henry pour vérifier l'hypothèse 4.

**Suggestion :** Vous pouvez utiliser la fonction `henryplot()` fournie pour tracer le diagramme quantile-quantile entre la loi normale et les résidus.


In [ ]:
function henryplot(y::Vector{<:Real})

    n = length(y)
    ysorted = sort(y)

    p = ( collect(1:n) .- .5 ) /n

    fd = fit(Normal,y)

    q = quantile.(fd,p)

    plot(x=ysorted, y=q, Geom.point,
    Guide.xlabel("Empirical quantiles"), Guide.ylabel("Estimated quantiles"),
    Theme(discrete_highlight_color=c->nothing),
    Geom.abline(color="red"))


end

In [ ]:
ŷ = X*β̂
e = y - ŷ

plot(x=ŷ, y=e, Geom.point, Guide.xlabel("ŷ"), Guide.ylabel("e"))

In [ ]:
henryplot(e)

## e) Relation pour les poids originaux

Tracez le nuage de points illutrants les poids originaux des perches en fonction votre variable explicative ainsi que la relation non linéaire obtenue.

**Suggestion :** Utilisez la fonction `layer()` pour superposer plusieurs couches sur un graphique.

In [ ]:
xx = collect(range(10, stop=50, length=1000))
yy = (β̂[1] .+ β̂[2]*xx).^3

obs = layer(perche, x=res.Variable[1], y=:Weight, Geom.point)
model = layer(x=xx, y=yy, Geom.line, Theme(default_color="red"))

Gadfly.set_default_plot_size(16cm, 9cm)
plot(obs, model)